In [6]:
#Loading the Dataset
import pandas as pd
from sklearn.impute import SimpleImputer
data = pd.read_csv('gptfile.csv')

#Pre-Prcoessing and Bag of Word Vectorization using Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(data['Sentences'])
#Splitting the data into trainig and testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, data['Sentiment'], test_size=0.25, random_state=5)
#Training the model
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(X_train, Y_train)
#Caluclating the accuracy score of the model
from sklearn import metrics
predicted = MNB.predict(X_test)
#print(predicted)
#accuracy_score = metrics.accuracy_score(predicted, Y_test)
#print("Accuracuy Score: ",accuracy_score),average='micro'
print("sample inputs:")
for i in range(3):
    print(i+1,": ",data['Sentences'][i])

y=MNB.predict(text_counts)
print("sample outputs:")
for i in range(3):
    print(i+1,": ",y[i])

sample inputs:
1 :  ChatGPT: Optimizing Language Models for Dialogue https://t.co/K9rKRygYyn @OpenAI
2 :  Try talking with ChatGPT, our new AI system which is optimized for dialogue. Your feedback will help us improve it. https://t.co/sHDm57g3Kr
3 :  ChatGPT: Optimizing Language Models for Dialogue https://t.co/GLEbMoKN6w #AI #MachineLearning #DataScience #ArtificialIntelligence\n\nTrending AI/ML Article Identified &amp; Digested via Granola; a Machine-Driven RSS Bot by Ramsey Elbasheer https://t.co/RprmAXUp34
sample outputs:
1 :  0
2 :  1
3 :  -1


In [7]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.model_selection import cross_val_score
y_pred=MNB.predict(X_test)
print('MultinomialNB Classifier')
print('accuracy :',accuracy_score(Y_test,y_pred))
print('F1 score :',f1_score(Y_test,y_pred,average='micro'))
print('Precision:',precision_score(Y_test,y_pred,average='micro'))
print('Recall   :',recall_score(Y_test,y_pred,average='micro'))
print('Cross validation score : \n',cross_val_score(estimator = MNB, X = X_test, y = Y_test, cv =5))
print('Mean of cross validation score : ',cross_val_score(estimator = MNB, X = X_test, y = Y_test, cv =5).mean())

MultinomialNB Classifier
accuracy : 0.73053772070626
F1 score : 0.73053772070626
Precision: 0.73053772070626
Recall   : 0.73053772070626
Cross validation score : 
 [0.69931601 0.68937528 0.7000456  0.69475604 0.70257205]
Mean of cross validation score :  0.6972129972102873


In [8]:
import nltk
#nltk.download('stopwords')
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import nltk
import re
import matplotlib.pyplot as plt
from spacy.lang.en.stop_words import STOP_WORDS
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
#Loading the dataset
data = pd.read_csv('finalDataset0.2.csv')
X=data["Sentences"]
Y=data["Sentiment"]
all_stopwords = stopwords.words('english')
all_stopwords.append("teacher")
all_stopwords.append("Ma'am")
all_stopwords.append("Maam")
all_stopwords.append("she")

print(all_stopwords)
for i in range(0,X.shape[0]):
    sample = X[i]
    lemmatizer = WordNetLemmatizer()
    sample = re.sub("[^a-zA-Z']", ' ', sample)
    sample = re.sub(r"can\'t",'can not',sample)
    sample = re.sub(r"n\'t", " not", sample)
    sample = sample.lower()
    sample = sample.split()
    sample = [lemmatizer.lemmatize(word) for word in sample if not word in set(all_stopwords)]
    sample = ' '.join(sample)
    X[i] = sample
le = LabelEncoder()
Y = le.fit_transform(Y)
dummy_y = to_categorical(Y)
print(dummy_y)
Y=dummy_y
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=5)
#print(X.head())
#print(X.shape)
#print(Y.shape)


max_words = 100000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train.values)
sequences = tok.texts_to_sequences(X_train.values)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)
#Y_train = np.array(Y_train)
#Y_test = np.array(Y_test) 
#Model Building
#'''
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(1024,name='FC1')(layer)
    layer = Dropout(0.5)(layer)
    layer = Activation('relu')(layer)
    layer = Dense(3,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

#'''
'''
    layer = Dropout(0.25)(layer)    
    layer = Dense(256,name='FC2')(layer)
    layer = Activation('softmax')(layer)
    
model = Sequential()
model.add(Bidirectional(LSTM(10, return_sequences=True),input_shape=(5, 10)))
model.add(Bidirectional(LSTM(10)))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
'''
# With custom backward layer
'''model = Sequential()
forward_layer = LSTM(10, return_sequences=True)
backward_layer = LSTM(10, activation='relu', return_sequences=True,go_backwards=True)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer,input_shape=(5, 10)))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
'''

#Model Training
history=model.fit(sequences_matrix,Y_train,batch_size=512,epochs=20,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.00000001)])
#Model Testing
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
#accr = model.evaluate(test_sequences_matrix,Y_test)
#print(accr)
#print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))




['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

C:\Users\kiran\AppData\Local\Temp\ipykernel_21180\54967986.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = sample


[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 200)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 200, 50)           5000000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 1024)              66560     
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 activation_2 (Activation)   (None, 1024)              0         
                                               

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

preds = model.predict(test_sequences_matrix) 
print(preds[0]) 
print(np.sum(preds[0])) 
matrix = confusion_matrix(Y_test.argmax(axis=1), preds.argmax(axis=1))
print(matrix)
# more detail on how well things were predicted
print(classification_report(Y_test.argmax(axis=1), preds.argmax(axis=1)))


9/9 [==============================] - 1s 30ms/step
[0.03456594 0.0493983  0.9160357 ]
0.99999994
[[  0   0  37]
 [  0   0  35]
 [  0   0 206]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        37
           1       0.00      0.00      0.00        35
           2       0.74      1.00      0.85       206

    accuracy                           0.74       278
   macro avg       0.25      0.33      0.28       278
weighted avg       0.55      0.74      0.63       278



c:\Users\kiran\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kiran\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kiran\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [10]:

print("sample inputs:")
for i in range(10):
    print(i+1,": ",data['Sentences'][i])

max_words = 100000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X.values)
tsequences = tok.texts_to_sequences(X.values)
tsequences_matrix = pad_sequences(sequences,maxlen=max_len)


y=model.predict(tsequences_matrix)
print("sample outputs:")
for i in range(10):
    print(i+1,": ",y.argmax(axis=1)[i]-1)

sample inputs:
1 :  punctual also give u practical knowledge theortical
2 :  good
3 :  excellent lecture delivered teacher teacher punctual
4 :  good
5 :  teacher give u information required improve performance
6 :  yes
7 :  good punctual
8 :  good
9 :  good
10 :  good
26/26 [==============================] - 1s 33ms/step
sample outputs:
1 :  1
2 :  1
3 :  1
4 :  1
5 :  1
6 :  1
7 :  1
8 :  1
9 :  1
10 :  1
